In [23]:
from math import *
from fractions import Fraction
import random

import numpy as np
import pandas as pd

from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Statevector
from qiskit_braket_provider import BraketLocalBackend

import qiskit.quantum_info as qi


class QuantumCircuitGenerator:

    hexColorArray = []
    gValColorArray = []
    thetaColorArray = []

    qubitCount = 0

    qCircuit = None
    encryptionCircuit = []
    encryptionKey = None

    def __init__(self, colorArray, encryptionKey) -> None:
        self.encryptionKey = encryptionKey
        self.hexColorArray = colorArray
        for hexColor in self.hexColorArray:
            hexColor = hexColor.lstrip("#")
            RGB = tuple(int(hexColor[i:i+2], 16) for i in (0, 2, 4))
            gVal = RGB[0]
            self.gValColorArray.append(gVal)
            theta = self.gVal2Theta(gVal)
            self.thetaColorArray.append(theta)
        self.qubitCount = len(self.hexColorArray)

    def generateQuantumCircuit(self):
        qCirc = QuantumCircuit(self.qubitCount)
        for pos in range(self.qubitCount):
            theta = self.gVal2Theta(self.thetaColorArray[pos])
            qCirc.ry(theta,pos)
        self.qCircuit = qCirc

    def gVal2Theta(self, gValue):
        return (gValue/255)*pi
    
    def generateEncryptionGate(self):
        self.encryptionCircuit = QuantumCircuit(len(self.encryptionKey))
        for index in range(len(self.encryptionKey)):
            match self.encryptionKey[index]:
                case "X":
                    self.encryptionCircuit.x(index)
                case "Y":
                    self.encryptionCircuit.y(index)
                case "Z":
                    self.encryptionCircuit.z(index)
                case "H":
                    self.encryptionCircuit.h(index)
                case "S":
                    self.encryptionCircuit.s(index)
                case "C":
                    self.encryptionCircuit.id(index)
        self.encryptionCircuit = self.encryptionCircuit.to_gate()
        self.qCircuit.append(self.encryptionCircuit,range(len(self.encryptionKey)))
    
    def infoChamp(self):
        op = qi.Operator(self.encryptionCircuit)
        print(op)

#~~~~~
c = QuantumCircuitGenerator(["#000000" for x in range(25)], [random.choice(["X","Y","Z","H","S","C"]) for x in range(25)])
c.generateQuantumCircuit()
c.generateEncryptionGate()
c.qCircuit.draw()


┌───────┐┌───────────────┐
 q_0: ┤ Ry(0) ├┤0              ├
      ├───────┤│               │
 q_1: ┤ Ry(0) ├┤1              ├
      ├───────┤│               │
 q_2: ┤ Ry(0) ├┤2              ├
      ├───────┤│               │
 q_3: ┤ Ry(0) ├┤3              ├
      ├───────┤│               │
 q_4: ┤ Ry(0) ├┤4              ├
      ├───────┤│               │
 q_5: ┤ Ry(0) ├┤5              ├
      ├───────┤│               │
 q_6: ┤ Ry(0) ├┤6              ├
      ├───────┤│               │
 q_7: ┤ Ry(0) ├┤7              ├
      ├───────┤│               │
 q_8: ┤ Ry(0) ├┤8              ├
      ├───────┤│               │
 q_9: ┤ Ry(0) ├┤9              ├
      ├───────┤│               │
q_10: ┤ Ry(0) ├┤10             ├
      ├───────┤│               │
q_11: ┤ Ry(0) ├┤11             ├
      ├───────┤│               │
q_12: ┤ Ry(0) ├┤12 circuit-197 ├
      ├───────┤│               │
q_13: ┤ Ry(0) ├┤13             ├
      ├───────┤│               │
q_14: ┤ Ry(0) ├┤14             ├
      ├───────┤│               │
q_15: ┤ Ry(0) ├┤15             ├
      ├───────┤│               │
q_16: ┤ Ry(0) ├┤16             ├
      ├───────┤│               │
q_17: ┤ Ry(0) ├┤17             ├
      ├───────┤│               │
q_18: ┤ Ry(0) ├┤18             ├
      ├───────┤│               │
q_19: ┤ Ry(0) ├┤19             ├
      ├───────┤│               │
q_20: ┤ Ry(0) ├┤20             ├
      ├───────┤│               │
q_21: ┤ Ry(0) ├┤21             ├
      ├───────┤│               │
q_22: ┤ Ry(0) ├┤22             ├
      ├───────┤│               │
q_23: ┤ Ry(0) ├┤23             ├
      ├───────┤│               │
q_24: ┤ Ry(0) ├┤24             ├
      └───────┘└───────────────┘